In [ ]:
##  Install ASAP1.9 and import
import os
import multiresolutionimageinterface as mir
import openslide
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage import io

In [ ]:
wsi_path = './wsi/'  ##  the location where wsis are stored
xml_path = './xml/'  ##  the location where region of interest (ROI) are stored
mask_path = './mask/'  ##   the location where ROI mask are stored
patch_path = './patch/'  ##   the location where patches are stored

In [ ]:
## Generate ROI mask
for Dir in os.listdir(wsi_path):
    if Dir.endswith('.mrxs'):          ##  modify according to the format of your scanner
        print(Dir)
        name = Dir.split('.')[0]
        #print(name)
        reader = mir.MultiResolutionImageReader()
        mr_image = reader.open(wsi_path + Dir)
        annotation_list = mir.AnnotationList()
        xml_repository = mir.XmlRepository(annotation_list)
        xml_repository.setSource(xml_path +name + '.xml')
        xml_repository.load()
        annotation_mask = mir.AnnotationToMask()
        output_path = mask_path + name + '.tif'
        annotation_mask.convert(annotation_list, output_path, mr_image.getDimensions(), mr_image.getSpacing())

In [ ]:
for Dir in os.listdir(wsi_path):
    if Dir.endswith('.mrxs'):
        print(Dir)
        name = Dir.split('.')[0]
        os.makedirs(patch_path + name) 
        img = openslide.OpenSlide(wsi_path + Dir)
        mask = openslide.OpenSlide(mask_path + name + '.tif')
        [w, h] = img.level_dimensions[1]
        img_gen = DeepZoomGenerator(img, tile_size=1024, overlap=0, limit_bounds=False)
        mask_gen = DeepZoomGenerator(mask, tile_size=1024, overlap=0, limit_bounds=False)
        
        width = highth = 1024     ##Set the resolution of a patch. If GPUs do NOT have enough memory, do NOT rxceed 256. 
        num_w = int(np.floor(w/width))+1
        num_h = int(np.floor(h/highth))+1
        for i in range(num_w):
            for j in range(num_h):
                seg = np.array(mask_gen.get_tile(img_gen.level_count-2, (i, j)))
                image = np.array(img_gen.get_tile(img_gen.level_count-2, (i, j)))
                
                if np.mean(seg) > 0.9:   ##Set the threshold for patch saving
                    io.imsave(os.path.join(patch_path + '/' + name ,name+'_'+str(i)+'_'+str(j)+".jpg"), image)